In [2]:
#This is the code for Table4.7 in Chapter4.3.2:The implementation and analysis of the strategy
#After modifying the dataset file path, it can be directly reproduce the results.

#!!! IMPORTANT REMINDER !!! 
#!!! IMPORTANT REMINDER !!! 
#!!! IMPORTANT REMINDER !!! 
# Before running this code, make sure to modify the input dataset path in the # === 1. Load and preprocess data ===


import pandas as pd
import numpy as np
from hmmlearn.hmm import GaussianHMM

# === 1. Load and preprocess data ===

#!!! IMPORTANT REMINDER !!! 
#!!! IMPORTANT REMINDER !!! 
#!!! IMPORTANT REMINDER !!! 
# !!! Modify "C:/Users/ZhangYinhang/ES_F_data.csv"  to your save path in the below !!!
df = pd.read_csv("C:/Users/ZhangYinhang/ES_F_data.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)
df['log_return'] = np.log(df['Close_ES=F'] / df['Close_ES=F'].shift(1))
df = df.dropna().reset_index(drop=True)

# === 2. Split training and testing sets ===
initial_end = pd.to_datetime("2024-12-23")
rolling_start = pd.to_datetime("2024-12-24")
train_df = df[df['Date'] <= initial_end].copy()
test_df = df[df['Date'] >= rolling_start].copy()

# === 3. Function to identify regimes by mean and std ===
def identify_states_by_mean_std(model):
    mu = model.means_.flatten()
    sigma = np.sqrt(np.array([np.diag(cov)[0] for cov in model.covars_]))
    labels = pd.DataFrame({'state': range(len(mu)), 'mu': mu, 'sigma': sigma})
    labels = labels.sort_values(by='mu')
    labels['regime'] = ['Bear', 'Neutral', 'Bull']
    return labels.set_index('state')['regime'].to_dict()

# === 4. Strategy simulation + model stability check ===
portfolio = []
portfolio_dates = []
window = 10
test_idx = 0

# Store model stability results
stability_records = []
prev_model = None
prev_state_map = None
prev_window_data = None
prev_window_regime = None

while test_idx + window <= len(test_df):
    # Train HMM on current window
    model = GaussianHMM(n_components=3, covariance_type="diag", n_iter=100, random_state=340)
    model.fit(train_df['log_return'].values.reshape(-1, 1))
    state_map = identify_states_by_mean_std(model)

    pred_slice = test_df.iloc[test_idx:test_idx + window].copy()
    pred_returns = pred_slice['log_return'].values.reshape(-1, 1)
    pred_states = model.predict(pred_returns)
    pred_slice['state'] = pred_states
    pred_slice['regime'] = pred_slice['state'].map(state_map)

    # Record strategy returns
    for _, row in pred_slice.iterrows():
        if row['regime'] == 'Bull':
            ret = np.log(row['Close_ES=F'] / row['Open_ES=F'])
        elif row['regime'] == 'Bear':
            ret = np.log(row['Open_ES=F'] / row['Close_ES=F'])
        else:
            ret = 0
        portfolio.append(ret)
        portfolio_dates.append(row['Date'])

    # === New: Check stability by re-evaluating previous window with current model ===
    if prev_model is not None:
        prev_returns = prev_window_data['log_return'].values.reshape(-1, 1)
        pred_states_prev_data = model.predict(prev_returns)
        pred_regimes_prev_data = pd.Series(pred_states_prev_data).map(state_map).values

        # Compute regime match rate
        match_rate = np.mean(pred_regimes_prev_data == prev_window_regime)
        stability_records.append({
            'PrevWindowStart': prev_window_data['Date'].iloc[0],
            'PrevWindowEnd': prev_window_data['Date'].iloc[-1],
            'CurrWindowStart': pred_slice['Date'].iloc[0],
            'CurrWindowEnd': pred_slice['Date'].iloc[-1],
            'RegimeMatchRate': match_rate
        })

    # Move rolling window forward
    prev_model = model
    prev_state_map = state_map
    prev_window_data = pred_slice.copy()
    prev_window_regime = pred_slice['regime'].values

    train_df = pd.concat([train_df, pred_slice], axis=0).reset_index(drop=True)
    test_idx += window

# === 5. Output model stability results ===
stability_df = pd.DataFrame(stability_records)
print("\n📊 Model Regime Stability Check Results:")
print(stability_df.to_string(index=False))



Model is not converging.  Current: 2024.7844793510455 is not greater than 2025.0014527775813. Delta is -0.21697342653578744
Model is not converging.  Current: 2059.6555311630236 is not greater than 2059.765338583978. Delta is -0.1098074209544393
Model is not converging.  Current: 2092.944704601734 is not greater than 2093.280412382798. Delta is -0.3357077810642295
Model is not converging.  Current: 2128.0581078789546 is not greater than 2128.062282524608. Delta is -0.0041746456531654985
Model is not converging.  Current: 2162.132663084039 is not greater than 2162.191796923344. Delta is -0.05913383930510463
Model is not converging.  Current: 2190.5392296913737 is not greater than 2190.7907005802354. Delta is -0.2514708888616042
Model is not converging.  Current: 2217.7463096163724 is not greater than 2218.0211406818194. Delta is -0.27483106544696057
Model is not converging.  Current: 2235.429334606658 is not greater than 2235.723247630315. Delta is -0.29391302365729643
Model is not conv


📊 Model Regime Stability Check Results:
PrevWindowStart PrevWindowEnd CurrWindowStart CurrWindowEnd  RegimeMatchRate
     2024-12-24    2025-01-08      2025-01-09    2025-01-23              1.0
     2025-01-09    2025-01-23      2025-01-24    2025-02-06              1.0
     2025-01-24    2025-02-06      2025-02-07    2025-02-21              1.0
     2025-02-07    2025-02-21      2025-02-24    2025-03-07              1.0
     2025-02-24    2025-03-07      2025-03-10    2025-03-21              1.0
     2025-03-10    2025-03-21      2025-03-24    2025-04-04              1.0
     2025-03-24    2025-04-04      2025-04-07    2025-04-21              1.0
     2025-04-07    2025-04-21      2025-04-22    2025-05-05              0.8
     2025-04-22    2025-05-05      2025-05-06    2025-05-19              1.0
     2025-05-06    2025-05-19      2025-05-20    2025-06-03              0.9
     2025-05-20    2025-06-03      2025-06-04    2025-06-17              1.0
